In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load Data

In [ ]:
df= pd.read_csv('/kaggle/input/car-price-prediction/CarPrice_Assignment.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df['Company']= df['CarName'].apply(lambda x: x.split(" ")[0])
df['Company']= df['Company'].apply(lambda x: x.lower())

In [ ]:
df.head()

In [ ]:
df['Company'].unique()

In [ ]:
df['Company'].replace({'maxda': 'mazda', 'porcshce':'porsche', 'toyouta':'toyota', 'vokswagen':'volkswagen','vw':'volkswagen'}, inplace=True)

In [ ]:
df['Company'].unique()

In [ ]:
df.drop(['car_ID','CarName'], axis=1, inplace=True)

In [ ]:
cat_cols= [col for col in df.columns if df[col].dtype=='object']
num_cols= [col for col in df.columns if df[col].dtype!='object']

In [ ]:
cat_cols

In [ ]:
num_cols
num_cols.remove('price')
num_cols

# Exploratory Data Analysis

In [ ]:
i=1
plt.figure(figsize=(30,100))
for col in cat_cols:
    plt.subplot(10,2,i)
    sns.countplot(df[col])
    plt.xticks(rotation=90, fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel(col, fontsize=15)
    plt.ylabel('count',fontsize=15)
    
    i+=1
    plt.subplot(10,2,i)
    sns.boxplot(x=df[col], y=df['price'])
    plt.xticks(rotation=90, fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel(col, fontsize=15)
    plt.ylabel('price',fontsize=15)
    i+=1
plt.show()

# Observations 
1. Diesel cars tend to be slightly higher-priced than gas cars
1. The number of gas cars is significantly higher than diesel cars
1. Turbo cars are higher priced than std cars
1. Sedan and hatchbacks account for more than 75% of total cars
1. Real wheel drive cars are higher priced than other drive cars
1. The median price of cars with engine at back is over 3 times the median price of cars with engine in front
1. Majority of the cars have ohc engines but, ohcv engines seem to be higher priced
1. General trend is that the price of car increase with increase in number of cylinders
1. Toyota is the most liked car
1. Porsche, Jaguar, BMW, Buick are high range cars

In [ ]:
i=1
plt.figure(figsize=(25,100))
for col in num_cols:
    plt.subplot(16,2,i)
    sns.distplot(df[col])
    plt.xlabel(col,fontsize=15)
    plt.xticks(fontsize=10)
    i+=1
    
    plt.subplot(16,2,i)
    sns.scatterplot(x=df[col], y=df['price'])
    plt.xlabel(col,fontsize=15)
    plt.xticks(fontsize=10)
    plt.ylabel('price', fontsize=15)
    plt.yticks(fontsize=10)
    i+=1
    

# Observations
1. Positive co-relation between price and car-length,car width, curb weight, engine size, horsepower
1. Negative co-relation between price and city mileage, highway mileage
1. There is no relation of symboling with price, hence we drop the column


In [ ]:
df.drop('symboling', axis=1, inplace=True)
num_cols.remove('symboling')

In [ ]:
df.head()

In [ ]:
num_cols

# Preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

df[cat_cols]= df[cat_cols].apply(lambda x: le.fit_transform(x))

In [ ]:
df[cat_cols].head()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
df[num_cols]= ss.fit_transform(df[num_cols])

In [ ]:
df.head()

# Training our Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X= df.drop('price', axis=1)
y=df['price']

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2, random_state= 42)

In [ ]:
lr= LinearRegression()
lr.fit(X_train,y_train)
y_pred= lr.predict(X_test)

rmse= (mean_squared_error(y_test,y_pred))**(1/2)
r2= r2_score(y_test,y_pred)

print(rmse)
print(r2)

In [ ]:
dt= DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=100, min_samples_leaf= 5, random_state=42)
dt.fit(X_train,y_train)
y_pred= dt.predict(X_test)

rmse_train= (mean_squared_error(dt.predict(X_train),y_train))**(1/2)
rmse= (mean_squared_error(y_test,y_pred))**(1/2)
r2= r2_score(y_test,y_pred)
print(rmse_train)
print(rmse)
print(r2)

In [ ]:
rf= RandomForestRegressor(max_depth=10, criterion='mse', min_samples_leaf=2, random_state=42, verbose=1)
rf.fit(X_train,y_train)
y_pred= rf.predict(X_test)
rmse_train= (mean_squared_error(rf.predict(X_train),y_train))**(1/2)
rmse= (mean_squared_error(y_test,y_pred))**(1/2)
r2= r2_score(y_test,y_pred)
print(rmse_train)
print(rmse)
print(r2)

In [ ]:
xgb= XGBRegressor()
xgb.fit(X_train, y_train)
y_pred= xgb.predict(X_test)
rmse_train= (mean_squared_error(xgb.predict(X_train),y_train))**(1/2)
rmse= (mean_squared_error(y_test,y_pred))**(1/2)
r2= r2_score(y_test,y_pred)
print(rmse_train)
print(rmse)
print(r2)

# Conclusion
All our models are overfitting the data. Tried many different combinations with GridSearchCV but there is still overfitting. This is because the amount of data is very less. However, with the given data, RandomForestRegressor is the best fit